In [1]:
import pandas as pd

df = pd.read_csv('takehome_user_engagement.csv')

In [2]:
df.set_index('time_stamp')
df['time_stamp'] = pd.to_datetime(df['time_stamp'])
df2 = df.groupby(['user_id',pd.Grouper(freq='1W',key = 'time_stamp')]).count()
# df.groupby(pd.Grouper(freq='1W',key = 'time_stamp')).count()

In [3]:
df_info = pd.read_csv('takehome_users.csv', encoding = 'latin1') 

In [4]:
df_info.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [5]:
creation_source_dummies = pd.get_dummies(df_info.creation_source).iloc[:,1:]
df_info = pd.concat([df_info, creation_source_dummies], axis=1)
df_info.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,0,0,0,0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,1,0,0,0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,1,0,0,0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,0,0,0,0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,0,0,0,0


In [6]:
df_info.set_index('object_id')
df_info['is_adopted'] = 0
for i in set(df2.index.get_level_values(0)):
    if max(df2.loc[i,'visited']) > 2: 
        df_info.loc[i-1,'is_adopted'] = 1

In [7]:
def invited(x):
    if x.endswith('INVITE') or x.endswith('JECTS'):
        return 1 
    else:
        return 0 
    
df_info['was_invited'] = df_info.creation_source.apply(invited)

In [8]:
org_value_counts = df_info.org_id.value_counts().to_dict()

df_info['org_size'] = df_info['org_id'].replace(org_value_counts)

In [9]:
import datetime

df_info['month'] = [datetime.datetime.strftime(pd.to_datetime(x), "%b") for x in df_info.creation_time.values]

creation_source_dummies = pd.get_dummies(df_info['month']).iloc[:,1:]
df_info = pd.concat([df_info, creation_source_dummies], axis=1)

In [10]:
X = df_info.drop(['object_id', 'name', 'creation_time', 'email', 'creation_source', 'month', 'last_session_creation_time', 'org_id','invited_by_user_id', 'is_adopted'], axis = 1)

In [11]:
Y = df_info[['is_adopted']]

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 57, stratify = Y)

In [16]:
import statsmodels.api as sm

logit = sm.Logit(y_train, X_train).fit()

Optimization terminated successfully.
         Current function value: 0.352809
         Iterations 7


In [17]:
logit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             is_adopted   No. Observations:                 9000
Model:                          Logit   Df Residuals:                     8981
Method:                           MLE   Df Model:                           18
Date:                Tue, 31 Dec 2019   Pseudo R-squ.:                 0.04078
Time:                        17:13:40   Log-Likelihood:                -3175.3
converged:                       True   LL-Null:                       -3310.3
Covariance Type:            nonrobust   LLR p-value:                 6.783e-47
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
opted_in_to_mailing_list       0.0677      0.085      0.794      0.427      -0.099       0.235
enabled_for_marketing_drip     0.0419      0.103      0.406      0.684      -0.160       0.244
ORG_INVITE                    -0.3040      0.090     -3.376      0.001      -0.480      -0.128
PERSONAL_PROJECTS             -0.8912      0.122     -7.322      0.000      -1.130      -0.653
SIGNUP                        -2.2253      0.150    -14.825      0.000      -2.519      -1.931
SIGNUP_GOOGLE_AUTH            -2.1196      0.158    -13.437      0.000      -2.429      -1.810
was_invited                   -2.0878      0.149    -14.044      0.000      -2.379      -1.796
org_size                      -0.0051      0.001     -7.096      0.000      -0.006      -0.004
Aug                            0.8584      0.170      5.037      0.000       0.524       1.192
Dec                            0.6280      0.174      3.616      0.000       0.288       0.968
Feb                            0.6608      0.173      3.809      0.000       0.321       1.001
Jan                            0.6739      0.168      4.003      0.000       0.344       1.004
Jul                            0.6675      0.177      3.766      0.000       0.320       1.015
Jun                            0.9057      0.170      5.318      0.000       0.572       1.240
Mar                            0.5433      0.168      3.226      0.001       0.213       0.873
May                           -0.5166      0.197     -2.619      0.009      -0.903      -0.130
Nov                            0.7996      0.170      4.713      0.000       0.467       1.132
Oct                            0.9400      0.168      5.599      0.000       0.611       1.269
Sep                            0.7943      0.170      4.663      0.000       0.460       1.128
==============================================================================================
"""

In [28]:
from sklearn.metrics import balanced_accuracy_score

threshold = [x/100 for x in range(-1,40)]

for i in threshold:
    Y_pred = (Y_prob > i).astype(int)
    print(i,f1_score(y_train, Y_pred ) , balanced_accuracy_score(y_train, Y_pred))

-0.01 0.21499404998016663 0.5
0.0 0.21499404998016663 0.5
0.01 0.21529294935451837 0.5008842849924204
0.02 0.21688194653048964 0.5056024139069081
0.03 0.22130725681935154 0.5185874654350383
0.04 0.22465522686598588 0.528035376993487
0.05 0.2318548387096774 0.5473202016001969
0.06 0.2380227246105189 0.5620530794747058
0.07 0.24277456647398846 0.5724892272924751
0.08 0.24861441013460017 0.5838196823100116
0.09 0.25826771653543307 0.6002987550087729
0.1 0.2618974516426159 0.6043521551941139
0.11 0.26507599280928257 0.6072760759189199
0.12 0.2670976522626744 0.6088635469477484
0.13 0.27064896755162243 0.6107943368468551
0.14 0.2735849056603774 0.6102717836172804
0.15 0.27068366164542296 0.6021796669457347
0.16 0.26471400939486045 0.5910773919512818
0.17 0.24714573539288118 0.5733546332431491
0.18 0.23243933588761176 0.5632646011907314
0.19 0.19806023481368043 0.5464692462740696
0.2 0.15600243753808654 0.5319435717095928
0.21 0.10359712230215828 0.5184301400871513
0.22 0.05320033250207813 0

In [31]:
Y_prob = logit.predict(X_test)
Y_pred = (Y_prob > 0.14).astype(int)
print(f1_score(y_test, Y_pred ) ,balanced_accuracy_score(y_test, Y_pred))

0.26103169670602855 0.5941933221998176


In [36]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, Y_pred))
print(classification_report(y_test, Y_pred))

[[1601 1038]
 [ 151  210]]
              precision    recall  f1-score   support

           0       0.91      0.61      0.73      2639
           1       0.17      0.58      0.26       361

    accuracy                           0.60      3000
   macro avg       0.54      0.59      0.50      3000
weighted avg       0.82      0.60      0.67      3000



These metrics were chosen due to the imbalanaced nature of the dataset. Both the F1 score and the balanced accuracy metric gave an optimal threshold of 0.14. The final results show a model that provides slight advantages in predicting user adoption, with a resultant f1 score of 0.26. The results are very close to the results from the training data, showing that we havent overfit our model. I tried to extract features that would illustrate core characteristics of the user rather than indicators which are inherently correlated with adoption (last login, number of login ..). These included method of invitation, whether they were invited by another user, size of their organization and which montht they signed up ( there could be seasonal effects). However, I would say while some correlations were found regarding month of sign up and method of sign up, we lack enough information about the users to be able to accurately differentiate the potential adopters from the non-adopters. 